# Cartopy tutorial: Transforming raster to vector

The most common raster to vector transformation is contouring. In addition to that though, there are a number of image processing techniques for identifying features of interest.


### Contouring
Let's start with contouring:

In [ ]:
import cartopy.crs as ccrs
from cartopy.examples.waves import sample_data
import matplotlib.pyplot as plt

lons, lats, data = sample_data()

ax = plt.axes(projection=ccrs.PlateCarree())
cs = plt.contourf(
    lons, lats, data, 5,
    transform=ccrs.PlateCarree())
ax.coastlines()
plt.colorbar(orientation='horizontal')
plt.show()

Let's pick out one of these contours. We could choose to pick out a line (``contour``) or a polygon (``contourf``).
Since we have already produced filled contours, let's use those:

In [ ]:
print(cs.levels)
print('N levels: {}'.format(len(cs.levels)))
print('N collections: {}'.format(len(cs.collections)))

In [ ]:
print(cs.levels[4:6])
print(cs.collections[4])
paths = cs.collections[4].get_paths()

We now need to convert these matplotlib Paths into Shapely geometries. The [``descartes``](https://pypi.org/project/descartes/) package is designed to do precisely this, but cartopy also has such functionality that has been tuned for performance:

In [ ]:
from cartopy.mpl.patch import path_to_geos
import shapely.ops

# Produces a list of shapely geometries.
geoms = []
for path in paths:
    geoms.extend(path_to_geos(path))

# Flatten the whole thing down to a single MultiPolygon
polygon = shapely.ops.unary_union(geoms)
polygon

We could use this geometry for any number of vector-type operations, but for now, let's just save it to disk. GeoJSON is a good choice for this kind of geometry - GeoJSON has become ubiquitous on the internet for its simpliciy and its interoperability with a number of popular tools/frameworks (including GitHub).

In [ ]:
import json
import shapely.geometry as sgeom

with open('contour.geojson', 'w') as fh:
    json.dump(sgeom.mapping(polygon), fh)

We've uploaded this geojson to a GitHub gist to show how GitHub renders this file: https://gist.github.com/pelson/837ca8c01d38157a2d634dcf97260e45

Be aware that the GeoJSON specification states that geometries MUST be saved in WGS84. The default PlateCarree instance is in fact equivalent to WGS84, so in this case we didn't need to convert the geometry. If had needed to convert it we could project the geometry with ``ccrs.PlateCarree().project_geometry(geometry, source_crs)``.

### Data space vs pixel space

So far we have used ``matplotlib.contour[f]`` to produce our contours, which conveniently returns contours in *data* coordinates. Sometimes we want to use tools that don't work in coordinate space, and which simply return pixel locations.

One of the most basic examples of this is numpy's ``where`` function. Let's use that to identify the yellow sections from the Wikipedia image below:

![Wikipedia route around the world](../../resources/640px-Around_the_World_in_Eighty_Days_map.png)

In [ ]:
import numpy as np
import skimage.io

# Image has shape: (y: 296, x: 640, RGBA: 4)
atw80d = skimage.io.imread(
    '../../resources/640px-Around_the_World_in_Eighty_Days_map.png')

yellowish = ((atw80d[:, :, 0] > 200) &  # Lots of Red.
             (atw80d[:, :, 1] > 200) &  # Lots of Green.
             (atw80d[:, :, 2] < 100) &  # Not lots of Blue.
             (atw80d[:, :, 3] > 250))   # Not transparent.

ind_y, ind_x = np.where(yellowish)
print('Number of yellow-ish pixels: ', len(ind_x))
print('x indexes: ', ind_x)
print('y indexes: ', ind_y)

Conversion between pixel locations and data locations is currently quite manual, but the following function will allow us to go from pixel centers to data coordinates:

In [ ]:
def pixels_to_data(extent, shape, i, j):
    """Converts from coordinates of the array to data coordinates."""
    xmin, xmax, ymin, ymax = extent
    x_range = xmax - xmin
    y_range = ymax - ymin
    
    pix_width = x_range / shape[1]
    pix_height = y_range / shape[0]

    # For y handle the fact that the image's pixels
    # start at the top (in mpl that is what origin='upper' means).
    j = shape[0] - j

    return (xmin + pix_width * (i + 0.5),
            ymin + pix_height * (j + 0.5))

If we feed in the image extent and the x and y indices computed using ``np.where``, we get the pixel location in the coordinate system of the image (Robinson):

In [ ]:
# Use the extents computed in an earlier exercise.
extent = [-13639040, 17156514, -6321250, 8621343]

# Note: Data coords in Robinson.
xs, ys = pixels_to_data(
    extent, atw80d.shape,
    ind_x, ind_y)

print('Last 3 x coordinates (Robinson): ', xs[-3:])
print('Last 3 y coordinates (Robinson): ', ys[-3:])

Now that we have the coordinates, let's plot the pixels we have identified:

In [ ]:
%matplotlib notebook
plt.figure()

rob = ccrs.Robinson(central_longitude=11.25)
ax = plt.axes(projection=rob)

ax.gridlines(color='gray', linestyle='--')
ax.coastlines()
ax.imshow(atw80d, extent=extent,
          transform=rob, origin='upper')
ax.set_global()

plt.plot(xs, ys, transform=rob,
         linestyle='none', marker='o',
         markeredgecolor='k', color='yellow')
plt.show()

Let's take this a step further and use skimage to pull out the track:

In [ ]:
# <insert Juan's work>

Exercise: Extract contours from a raster, save to GeoJSON.

*Exercise:* Pull out the route from the wikipedia image.

*Exercise:* Project the geometry to PC, so that we can save it to GeoJSON.